In [ ]:
# Function to calculate the mean of a list
def calculate_mean(data):
    return sum(data) / len(data)

# Function to calculate the covariance matrix
def calculate_covariance_matrix(data):
    num_features = len(data[0])
    num_samples = len(data)
    mean_vector = [calculate_mean(feature) for feature in zip(*data)]
    covariance_matrix = [[0]*num_features for _ in range(num_features)]
    for sample in data:
        for i in range(num_features):
            for j in range(num_features):
                covariance_matrix[i][j] += (sample[i] - mean_vector[i]) * (sample[j] - mean_vector[j]) / (num_samples - 1)
    return covariance_matrix

# Function to perform matrix multiplication
def matrix_multiply(matrix1, matrix2):
    result = []
    for i in range(len(matrix1)):
        row = []
        for j in range(len(matrix2[0])):
            element = 0
            for k in range(len(matrix2)):
                element += matrix1[i][k] * matrix2[k][j]
            row.append(element)
        result.append(row)
    return result

# Function to calculate the mean vector of each class
def calculate_class_means(data, labels):
    class_means = {}
    for sample, label in zip(data, labels):
        if label not in class_means:
            class_means[label] = [sample]
        else:
            class_means[label].append(sample)
    for label, samples in class_means.items():
        class_means[label] = [calculate_mean(feature) for feature in zip(*samples)]
    return class_means

# Function to calculate the within-class scatter matrix
def calculate_within_class_scatter_matrix(data, labels):
    num_features = len(data[0])
    class_means = calculate_class_means(data, labels)
    within_class_scatter_matrix = [[0]*num_features for _ in range(num_features)]
    for label, samples in zip(labels, data):
        for i in range(num_features):
            for j in range(num_features):
                within_class_scatter_matrix[i][j] += (samples[i] - class_means[label][i]) * (samples[j] - class_means[label][j])
    return within_class_scatter_matrix

# Function to calculate the between-class scatter matrix
def calculate_between_class_scatter_matrix(data, labels):
    num_features = len(data[0])
    overall_mean = [calculate_mean(feature) for feature in zip(*data)]
    class_means = calculate_class_means(data, labels)
    between_class_scatter_matrix = [[0]*num_features for _ in range(num_features)]
    for label, samples in class_means.items():
        for i in range(num_features):
            for j in range(num_features):
                between_class_scatter_matrix[i][j] += len(samples) * (class_means[label][i] - overall_mean[i]) * (class_means[label][j] - overall_mean[j])
    return between_class_scatter_matrix

# Function to find eigenvalues and eigenvectors of a matrix
def find_eigen(matrix):
    eigenvalues = []
    eigenvectors = []
    for i in range(len(matrix)):
        eigenvalue, eigenvector = power_iteration(matrix, 1000)
        eigenvalues.append(eigenvalue)
        eigenvectors.append(eigenvector)
        # Deflate the matrix
        for j in range(len(matrix)):
            for k in range(len(matrix[0])):
                matrix[j][k] -= eigenvalue * eigenvector[j] * eigenvector[k]
    return eigenvalues, eigenvectors

# Function to perform Linear Discriminant Analysis (LDA)
def lda(data, labels, num_components):
    within_class_scatter_matrix = calculate_within_class_scatter_matrix(data, labels)
    between_class_scatter_matrix = calculate_between_class_scatter_matrix(data, labels)
    
    # Find eigenvalues and eigenvectors of the matrix inverse of within-class scatter matrix times between-class scatter matrix
    inverse_within_class_scatter_matrix = [[1/within_class_scatter_matrix[i][j] if i == j else 0 for j in range(len(within_class_scatter_matrix[0]))] for i in range(len(within_class_scatter_matrix))]
    product_matrix = matrix_multiply(inverse_within_class_scatter_matrix, between_class_scatter_matrix)
    eigenvalues, eigenvectors = find_eigen(product_matrix)
    
    # Select the top num_components eigenvectors corresponding to the largest eigenvalues
    sorted_indices = sorted(range(len(eigenvalues)), key=lambda i: eigenvalues[i], reverse=True)
    selected_indices = sorted_indices[:num_components]
    selected_eigenvectors = [eigenvectors[i] for i in selected_indices]
    
    # Project the data onto the selected eigenvectors
    projected_data = matrix_multiply(data, transpose(selected_eigenvectors))
    
    return projected_data

# Example data and labels
data = [
    [1, 2],
    [2, 3],
    [3, 4],
    [4, 5],
    [5, 6]
]
labels = [0, 1, 0, 1, 0]

# Perform Linear Discriminant Analysis (LDA)
num_components = 1
projected_data = lda(data, labels, num_components)

# Displaying the projected data
print("Projected Data:")
for sample in projected_data:
    print(sample)
